In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# | hide
from onprem.core import *

# Guided Prompts



The `Guider` in **OnPrem.LLM**, a simple interface to the Guidance package, can be used to guide the output of an LLM based on conditions and constraints that you supply.  

Let's begin by creating an `onprem.LLM` instance.

In [ ]:
# | notest
from onprem import LLM
llm = LLM(n_gpu_layers=35) # set based on your system

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 2 CUDA devices:
  Device 0: NVIDIA TITAN V, compute capability 7.0
  Device 1: NVIDIA TITAN V, compute capability 7.0
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /home/amaiya/onprem_data/Wizard-Vicuna-7B-Uncensored.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.wei

Next, let's create a `Guider` instance.

In [ ]:
# | notest

from onprem.guider import Guider
guider = Guider(llm)

The `guider.prompt` method accepts [Guidance](https://github.com/guidance-ai/guidance) prompts as input. (You can refer to the [Guidance documentation](https://github.com/guidance-ai/guidance) for information on how to construct such prompts (also called Guidance programs).

Here, we'll show some examples (mostly taken from the Guidance documentation) and begin with importing some Guidance functions.



## The `select` function

The `select` function allows you to guide the LLM generate output from only a finite set of alternatives. The `Guider.prompt` method returns a dictionary with the answer associated with the key you supply in the prompt.

In [ ]:
# | notest
from guidance import select

In [ ]:
# | notest
guidance_program = f'Do you want a joke or a poem? A ' + select(['joke', 'poem'], name='answer') # example from Guidance documentation
guider.prompt(guidance_program)

{'answer': 'joke'}

## The `gen` function

The `gen` function allows you to place conditions and constraints on the generated output.

In [ ]:
# | notest

from guidance import gen

In [ ]:
# | notest
guider.prompt(f'The capital of France is {gen("answer", max_tokens=1, stop=".")}')

{'answer': 'Paris'}

You can also use regular expressions to guide the output.

In [ ]:
# | notest

prompt = f"""Question: Luke has ten balls. He gives three to his brother. How many balls does he have left?
Answer: """ + gen('answer', regex='\d+')

In [ ]:
# | notest

guider.prompt(prompt)

{'answer': '7'}

In [ ]:
# | notest

prompt = '19, 18,' + gen('answer', max_tokens=50, stop_regex='[^\d]7[^\d]')
guider.prompt(prompt)

{'answer': ' 17, 16, 15, 14, 13, 12, 11, 10, 9, 8,'}

## Structured Outputs

Using `select` and `gen`, you can guide the LLM to produce outputs conforming to the structure that you want (e.g., JSON).

Let's create a prompt for generating fictional D&D-type characters.

In [ ]:
# | notest


sample_weapons = ["sword", "axe", "mace", "spear", "bow", "crossbow"]
sample_armour = ["leather", "chainmail", "plate"]

def generate_character_prompt(
    character_one_liner,
    weapons: list[str] = sample_weapons,
    armour: list[str] = sample_armour,
    n_items: int = 3
):

    prompt = ''
    prompt += "{"
    prompt += f'"description" : "{character_one_liner}",'
    prompt += '"name" : "' + gen(name="character_name", stop='"') + '",'
    # With guidance, we can call a GPU rather than merely random.randint()
    prompt += '"age" : ' + gen(name="age", regex="[0-9]+") + ','
    prompt += '"armour" : "' + select(armour, name="armour") + '",'
    prompt += '"weapon" : "' + select(weapons, name="weapon") + '",'
    prompt += '"class" : "' + gen(name="character_class", stop='"') + '",'
    prompt += '"mantra" : "' + gen(name="mantra", stop='"') + '",'
    # Again, we can avoid calling random.randint() like a pleb
    prompt += '"strength" : ' + gen(name="age", regex="[0-9]+") + ','
    prompt += '"quest_items" : [ '
    for i in range(n_items):
        prompt += '"' + gen(name="items", list_append=True, stop='"') + '"'  
        # We now pause a moment to express our thoughts on the JSON
        # specification's dislike of trailing commas
        if i < n_items - 1:
            prompt += ','
    prompt += "]"
    prompt += "}"
    return prompt

In [ ]:
# | notest


d = guider.prompt(generate_character_prompt("A quick and nimble fighter"))

### The Generated Dictionary:

In [ ]:
# | notest

d

{'items': ['Quest Item 3', 'Quest Item 2', 'Quest Item 1'],
 'age': '10',
 'mantra': 'I am the blade of justice.',
 'character_class': 'fighter',
 'weapon': 'sword',
 'armour': 'leather',
 'character_name': 'Katana'}

### Convert to JSON

In [ ]:
# | notest

import json
print(json.dumps(d, indent=4))

{
    "items": [
        "Quest Item 3",
        "Quest Item 2",
        "Quest Item 1"
    ],
    "age": "10",
    "mantra": "I am the blade of justice.",
    "character_class": "fighter",
    "weapon": "sword",
    "armour": "leather",
    "character_name": "Katana"
}
